<a href="https://colab.research.google.com/github/ChinthakindhiNishitha/ML-LAB/blob/main/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from pathlib import Path

In [2]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd


In [5]:
df1=pd.read_csv('/content/drive/MyDrive/titanic_train.csv')
df1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
df1.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
df1=df1.drop(['PassengerId','Name','Ticket'],axis=1)

In [8]:
df1.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [9]:
[col for col in df1.columns if df1[col].isnull().sum()>0]

['Age', 'Cabin', 'Embarked']

In [10]:
df1['Age'] = df1['Age'].fillna(df1['Age'].median())

In [11]:
df1=df1.drop(['Embarked','Cabin'],axis=1)

In [12]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
X = df1.drop('Survived',axis=1)
y = df1['Survived']
X = pd.get_dummies(X,drop_first=True)

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)


In [13]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

0


In [14]:
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)


In [15]:
sfs1=sfs1.fit(np.array(x_train.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.3s finished

[2025-08-20 08:17:51] Features: 1/5 -- score: 0.765194136612553[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.1s finished

[2025-08-20 08:17:55] Features: 2/5 -- score: 0.832223720877483[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.8s finished

[2025-08-20 08:17:59] Features: 3/5 -- score: 0.8401143433459706[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.6s finished

[2025-08-20 08:18:04] Features: 4/5 -- score: 0.842493964874761[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.7s finished

[2025-08-20 08:18:05] Features: 5/5 -- score: 0.8489306892194032

In [16]:
print('best accracy score',sfs1.k_score_)
print('best subset(indices)',sfs1.k_feature_idx_)
print('Best subset (corresponding names):', sfs1.k_feature_names_)

best accracy score 0.8489306892194032
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


In [17]:
sfs2=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)
sfs2=sfs2.fit(np.array(x_train.fillna(0)),y_train)
print('best accracy score',sfs2.k_score_)
print('best subset(indices)',sfs2.k_feature_idx_)
print('Best subset (corresponding names):', sfs2.k_feature_names_)

best accracy score 0.8452298208094359
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.0s finished

[2025-08-20 08:18:25] Features: 5/5 -- score: 0.8452298208094359

In [18]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
efs=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=5)
efs=efs.fit(x_train,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

Features: 56/56

best accuracy score 0.8154709677419355
best subset(indices) (0, 1, 2, 5)
Best subset (corresponding names): ('Pclass', 'Age', 'SibSp', 'Sex_male')


In [19]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(1,)","[0.608, 0.568, 0.624, 0.5725806451612904, 0.58...",0.590645,"(Age,)",0.027881,0.021693,0.010846
12,"(1, 3)","[0.56, 0.568, 0.632, 0.6209677419354839, 0.612...",0.598774,"(Age, Parch)",0.037457,0.029143,0.014572
13,"(1, 4)","[0.616, 0.6, 0.6, 0.5483870967741935, 0.645161...",0.60191,"(Age, Fare)",0.040405,0.031436,0.015718
3,"(3,)","[0.568, 0.6, 0.624, 0.6048387096774194, 0.6451...",0.6084,"(Parch,)",0.03307,0.02573,0.012865
34,"(1, 3, 4)","[0.576, 0.624, 0.616, 0.5967741935483871, 0.63...",0.609974,"(Age, Parch, Fare)",0.027549,0.021434,0.010717
11,"(1, 2)","[0.648, 0.576, 0.648, 0.5645161290322581, 0.62...",0.611497,"(Age, SibSp)",0.04534,0.035276,0.017638
31,"(1, 2, 3)","[0.648, 0.576, 0.672, 0.5887096774193549, 0.63...",0.624361,"(Age, SibSp, Parch)",0.046698,0.036333,0.018166
51,"(1, 2, 3, 4)","[0.664, 0.648, 0.624, 0.6048387096774194, 0.62...",0.632361,"(Age, SibSp, Parch, Fare)",0.026978,0.020989,0.010495
2,"(2,)","[0.624, 0.624, 0.656, 0.5645161290322581, 0.71...",0.637252,"(SibSp,)",0.064192,0.049944,0.024972
32,"(1, 2, 4)","[0.672, 0.64, 0.656, 0.5887096774193549, 0.661...",0.6436,"(Age, SibSp, Fare)",0.037689,0.029323,0.014661


In [20]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0, criterion='gini')
rfc.fit(x_train, y_train)
for feature in zip(x_train.columns, rfc.feature_importances_):
    print(feature)

('Pclass', np.float64(0.08580548090109545))
('Age', np.float64(0.26433757901109173))
('SibSp', np.float64(0.05355992352286061))
('Parch', np.float64(0.03699908167792097))
('Fare', np.float64(0.28790097031129963))
('Sex_male', np.float64(0.2713969645757317))


In [21]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(estimator=rfc, threshold=0.2)
sfm = sfm.fit(x_train, y_train)
feature_names=x_train.columns
print('The most important features based on random forest classifier:')
for feature_list_index in sfm.get_support(indices=True):
    print('- ' + feature_names[feature_list_index])

The most important features based on random forest classifier:
- Age
- Fare
- Sex_male


In [22]:
from sklearn.metrics import accuracy_score
rfc_full = RandomForestClassifier(random_state=0, criterion='gini')
rfc_full.fit(x_train, y_train)
pred_full = rfc_full.predict(x_test)
print('The accuracy of classifier with full features: {:.2f}'.format(accuracy_score(y_test, pred_full)))


The accuracy of classifier with full features: 0.82
